In [1]:
%pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [164]:
import os.path
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat

subjects = [102, 104, 105, 107, 109, 110, 111, 115, 116, 117, 118, 120, 126, 127, 130, 131, 132, 133, 135, 138, 141, 143, 144]
col = ['1','2','3','Label', 'Frontal P3 mean', 'Frontal P3 STD', 'Posterior P3 mean', 'Posterior P3 STD', 'Frontal alpha mean', 
           'Posterior alpha mean', 'Alpha variability', 'Reaction time Mean', 'Reaction time variability', 'Accuracy', 'Frontal P3 log energy entropy', 
           'Frontal P3 Shannon entropy', 'Frontal P3 SURE entropy', 'Frontal P3 Skewness', 'Frontal P3 Kurtosis', 'Frontal alpha log energy entropy',
           'Frontal alpha Shannon entropy', 'Frontal alpha SURE entropy', 'Frontal alpha Skewness', 'Frontal alpha Kurtosis', 
           'Posterior P3 log energy entropy', 'Posterior P3 Shannon entropy', 'Posterior P3 SURE entropy', 'Posterior P3 Skewness', 'Posterior P3 Kurtosis', 
           'Posterior alpha log energy entropy', 'Posterior alpha Shannon entropy', 'Posterior alpha SURE entropy', 'Posterior alpha Skewness',
           'Posterior alpha Kurtosis'
]

#Load all subject mat files, append TR and TUR structures to dataframe
for a in subjects:
    file = 'Feature_data_'+str(a)
    loc = os.path.join('C:/Users/pisis/OneDrive - University of Calgary/2024/AIRS/TR and TUT data',file)
    subData = loadmat(loc)['data']
    subDataTR = subData['TR'][0,0]
    subDataTUR = subData['TUR'][0,0]
    subDFTR = pd.DataFrame(subDataTR, columns = col)
    subDFTUR = pd.DataFrame(subDataTUR, columns = col)
    if a==subjects[0]:
        totalDF = pd.concat([subDFTR,subDFTUR])
    else:
        totalDF = pd.concat([totalDF, subDFTR])
        totalDF = pd.concat([totalDF, subDFTUR])
print(totalDF)

        1     2    3  Label  Frontal P3 mean  Frontal P3 STD  \
0   102.0   8.0  4.0    1.0        -4.473627        0.576930   
1   102.0  25.0  5.0    1.0        -1.786179        0.747948   
2   102.0  30.0  5.0    1.0         2.321102        0.274870   
3   102.0  32.0  4.0    1.0        -2.569933        0.658272   
0   102.0   1.0  5.0    2.0         2.814653        0.359229   
..    ...   ...  ...    ...              ...             ...   
19  144.0  30.0  3.0    2.0         0.375386        0.774057   
20  144.0  31.0  3.0    2.0         0.482558        2.200203   
21  144.0  32.0  3.0    2.0        -0.893606        0.333826   
22  144.0  34.0  3.0    2.0        -1.408621        0.136761   
23  144.0  35.0  1.0    2.0        -2.785345        0.359457   

    Posterior P3 mean  Posterior P3 STD  Frontal alpha mean  \
0            0.511275          0.849015            0.424295   
1            1.117962          0.297814            2.976309   
2           -0.140859          0.650573   

In [169]:
cleanData = totalDF.iloc[:,3:]
# print(cleanData)
# print(cleanData.Label.unique())

[1. 2.]


In [ ]:
from imblearn.over_sampling import SMOTE